# DA00_Function_Import_Main_df

This notebook demonstrates the example execution of `DA00_Function_Import_Main_df.py`. We will go step by step through its functions, explaining their usage with examples.

## Data Import

### Import multiple Neware txt files, loop until all files found, remove tab and header using skiprows, remove index and "." as decimal

In [ ]:

def DA00_Function_Import(data_folder,file_name,rated_capacity):
        df_main = []
    i = 0  # Start counter

    while True:
        file_path = f"{data_folder}/{file_name}/{file_name}__{i}.txt"
        if os.path.exists(file_path):
            dataraw = pd.read_csv(file_path, sep='\t', skiprows=0, index_col=False, decimal='.')
            df_main.append(dataraw)  # Append dataframe to the list
            i += 1  # Increment counter for the next file
        else:
            print(f"File {file_path} not found. Stopping.")
            break  # Exit loop when the file is not found

### Combine into one main dataframe and renaming to prevent further error

In [ ]:
    df_main = pd.concat(df_main, ignore_index=True)
    name_conversion_dict = {'Time(h:min:s.ms)': 'Time', 'Voltage(V)':'Voltage', 'Current(mA)': 'Current', 'Capacity(mAh)': 'Capacity', 'dQ/dV(mAh/V)':'dQdV'}
    df_main = df_main.rename(columns=name_conversion_dict)
    df_main['Realtime'] = pd.to_datetime(df_main['Realtime'], format='%m/%d/%Y %H:%M:%S')
    df_main['Time'] = pd.to_timedelta(df_main['Time'].astype(str))
    df_main['Cycle_Time'] = df_main.groupby('Cycle ID')['Time'].cumsum()
    df_main['SOH'] = df_main['Capacity']/rated_capacity
    
    return df_main

## Data Grouping

In [ ]:
def DA00_Function_df_Cycle_Grouping(df_main,result_folder,file_name):
    #Make result folder
    os.makedirs(f"{result_folder}/{file_name}", exist_ok=True)  # exist_ok=True avoids errors if the folder already exists
    print(f"Folder '{result_folder}/{file_name}' created!")

# Function to add CC_Chg to CV_Chg capacity (as initial capacity)

In [ ]:
    def accumulate_capacity(df, start_capacity):
        df['Accumulated_Capacity'] = df['Capacity'] + start_capacity
        return df

# Grouping dataframe by Cycle ID, cycle_id = the cycle numbers

In [ ]:
    df_cycle_grouped = df_main.groupby('Cycle ID')
    cycle_id = df_cycle_grouped.groups.keys()
    ## Initialize an empty list to store data for each cycle

### Group by Step Name within each cycle & Iterate over each cycle

In [ ]:
    df_VQ_grouped_list = []
    for cycle_id, df_cyc in df_cycle_grouped:
        df_step_grouped = df_cyc.groupby('Step Name')
        step_group_name = df_step_grouped.groups.keys()
       
        if 'CC_Chg' and 'CV_Chg' and 'CC_DChg' in step_group_name:
            cc_chg = df_step_grouped.get_group('CC_Chg') if 'CC_Chg' in df_step_grouped.groups else pd.DataFrame()
            cv_chg = df_step_grouped.get_group('CV_Chg') if 'CV_Chg' in df_step_grouped.groups else pd.DataFrame()
            cc_dchg = df_step_grouped.get_group('CC_DChg') if 'CC_DChg' in df_step_grouped.groups else pd.DataFrame()
            vchg = pd.concat([cc_chg['Voltage'], cv_chg['Voltage']]) if not cv_chg.empty else cc_chg['Voltage']
            dqdvchg = pd.concat([cc_chg['dQdV'], cv_chg['dQdV']]) if not cv_chg.empty else cc_chg['dQdV']
            cchg_start = cc_chg['Capacity'].iloc[-1] if not cc_chg.empty else 0
            cchg = pd.concat([cc_chg['Capacity'], accumulate_capacity(cv_chg, cchg_start)['Accumulated_Capacity']]) if not cv_chg.empty else   cc_chg['Capacity']
            vdchg = cc_dchg['Voltage'] if not cc_dchg.empty else pd.Series()
            cdchg = cc_dchg['Capacity'] if not cc_dchg.empty else pd.Series()
            dqdvdchg = cc_dchg['dQdV'] if not cc_dchg.empty else pd.Series()

### Combine into a DataFrame for V-Q

In [ ]:
            df_VQ = pd.DataFrame({
                f'Cycle_{cycle_id}_VChg': vchg.reset_index(drop=True),
                f'Cycle_{cycle_id}_CapChg': cchg.reset_index(drop=True),
                f'Cycle_{cycle_id}_dQdVChg': dqdvchg.reset_index(drop=True),
                f'Cycle_{cycle_id}_VDChg': vdchg.reset_index(drop=True),
                f'Cycle_{cycle_id}_CapDChg': cdchg.reset_index(drop=True),
                f'Cycle_{cycle_id}_dQdVDChg': dqdvdchg.reset_index(drop=True),
                })
            df_VQ_grouped_list.append(df_VQ)
        
        else:
            continue

### Concatenate all cycle data horizontally and save the result

In [ ]:
    df_VQ_grouped = pd.concat(df_VQ_grouped_list, axis=1)
    df_VQ_grouped.to_csv(f'{result_folder}/{file_name}/df_VQ_grouped_{file_name}.csv', index=False)

return df_cycle_grouped,df_VQ_grouped